In [1]:
!git clone https://github.com/wangyirui/AD_Prediction.git

Cloning into 'AD_Prediction'...
remote: Enumerating objects: 594, done.
remote: Total 594 (delta 0), reused 0 (delta 0), pack-reused 594
Receiving objects: 100% (594/594), 5.71 MiB | 16.82 MiB/s, done.
Resolving deltas: 100% (357/357), done.


In [ ]:
!cp -R /content/drive/MyDrive/Data/* .

In [ ]:
cd ..

/content


In [ ]:
!unzip 'ADNI_TEST/ADNI*.zip'

Archive:  ADNI_TEST/ADNI1:Baseline 3T4.zip
  inflating: ADNI/023_S_0388/MPR__GradWarp__B1_Correction__N3__Scaled/2006-05-04_15_27_08.0/I31446/ADNI_023_S_0388_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20061203120855584_S14001_I31446.nii  
  inflating: ADNI/005_S_0448/MPR-R__GradWarp__B1_Correction__N3__Scaled/2006-05-22_15_18_16.0/I32785/ADNI_005_S_0448_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20061212143206036_S14868_I32785.nii  
  inflating: ADNI/127_S_0397/MPR__GradWarp__B1_Correction__N3__Scaled/2006-06-06_12_07_37.0/I34416/ADNI_127_S_0397_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20061228110515922_S15283_I34416.nii  
  inflating: ADNI/068_S_0478/MPR-R____N3__Scaled/2006-07-12_10_20_53.0/I37041/ADNI_068_S_0478_MR_MPR-R____N3__Scaled_Br_20070120022202561_S16782_I37041.nii  
  inflating: ADNI/130_S_0886/MPR-R____N3__Scaled/2006-10-16_08_49_47.0/I39288/ADNI_130_S_0886_MR_MPR-R____N3__Scaled_Br_20070211002657559_S20151_I39288.nii  
  inflating: ADNI/136_S_0426/MPR____N

In [ ]:
cd ADNI

/content/ADNI


In [ ]:
import os
import glob
import numpy as np

In [ ]:
ls . | wc -l

151


In [ ]:
# for subject in os.listdir():
len(glob.glob('*/*/*/*/*.nii'))

199

In [ ]:
import nibabel as nib

In [ ]:
import numpy as np
from scipy.ndimage import zoom

def resize_mri(image, target_shape):
    current_shape = np.array(image.shape)
    target_shape = np.array(target_shape)

    # Calculate the scaling factor for each dimension
    scale_factors = target_shape / current_shape

    # Perform interpolation using zoom to resize the image
    resized_image = zoom(image, scale_factors, mode='nearest')

    return resized_image


In [ ]:
sizes = set()

In [ ]:
count = 0

In [ ]:
for file_path in glob.glob('*/*/*/*/*.nii'):
  nifti_img = nib.load(file_path)

  # Load the NIfTI file
  nifti_img = nib.load(file_path)

  # Get the image data and header
  img_data = nifti_img.get_fdata()
  print(img_data.dtype)
  img_header = nifti_img.header

  # Get the dimensions (shape) of the image
  dimensions = img_data.shape
  # print(type(img_data))
  # sizes.add(dimensions)
  if dimensions == (240, 256, 160):
    continue
  count+=1


  # Print the dimensions
  # print(f"Dimensions: {dimensions}\t {file_path}")

In [ ]:
mkdir ../ADNI_PREP

In [ ]:
subjects_completed = set()

In [ ]:
for file_path in glob.glob('*/*/*/*/*.nii'):

  subject = file_path.split(os.path.sep)[0]

  if subject in subjects_completed:
    continue
  else:
    subjects_completed.add(subject)

  # Load the original NIfTI file
  original_nifti = nib.load(file_path)

  # Access the data array from the NIfTI image
  data = original_nifti.get_fdata()

  # Perform dimension standardization on the data array (replace this with your own logic)
  # For example, you can resize the array to a target shape using numpy's zoom function
  target_shape = (256, 256, 160)
  resized_data = resize_mri(np.array(data), target_shape)

  # Create a memory-mapped array from the resized data
  resized_memmap = np.memmap('resized_memmap.dat', dtype=resized_data.dtype, mode='w+', shape=resized_data.shape)
  resized_memmap[:] = resized_data

  # Create a new NIfTI image with the standardized data
  standardized_nifti = nib.Nifti1Image(resized_memmap, affine=original_nifti.affine)

  # Save the modified NIfTI file to a new location
  output_path = os.path.join('..', 'ADNI_PREP', subject + '.nii')
  nib.save(standardized_nifti, output_path)

  # Remember to close the memmap object
  del resized_memmap

In [ ]:
cd ..

/content


In [ ]:
ls ADNI_PREP | wc -l

151


In [ ]:
# !zip preped_test.zip ADNI_PREP/* 1>/dev/null
# !tar -czvf preped_test.tar.gz ADNI_PREP/* 1>/dev/null

In [ ]:
# !cp preped_test.zip /content/drive/MyDrive/Data/

In [ ]:
import os
subjects = set()
with open('2c_classes.txt', 'w') as f, open('classes.txt', 'r') as r:
  for line in r.readlines():
    subject = line.split('\t')[0]
    if subject in os.listdir('ADNI_PREP') and subject not in subjects:
      f.write(line)
      subjects.add(subject)


In [ ]:
train = 0.8

with open('2c_classes.txt', 'r') as f, open('2c_train_classes.txt', 'w') as w1, open('2c_test_classes.txt', 'w') as w2:
  samples = f.readlines()
  sample_count = len(samples)
  for i, line in enumerate(samples):
    if i < train * sample_count:
      w1.write(line)
    else:
      w2.write(line)


In [ ]:
!mv ADNI_PREP/ AD_Prediction/Image/

In [ ]:
!cp 2c_train_classes.txt AD_Prediction/train_2classes.txt

In [ ]:
!cp 2c_test_classes.txt AD_Prediction/train_2classes.txt

In [ ]:
cd AD_Prediction/

/content/AD_Prediction


In [ ]:
!python main_alexnet.py --optimizer Adam --learning_rate 1e-4 --save AlexNet-fine-tune-fc-last-conv-lr1e-4 --batch_size 16 --epochs 200 --gpuid 0

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth
100% 233M/233M [00:01<00:00, 175MB/s]
2023-12-03 20:39:16 INFO: At 0-th epoch.
Traceback (most recent call last):
  File "/content/AD_Prediction/main_alexnet.py", line 238, in <module>
    main(options)
  File "/content/AD_Prediction/main_alexnet.py", line 134, in main
    train_loss, correct_cnt = train(model, train_loader, use_cuda, criterion, optimizer, train_loss_f)
  File "/cont